Ejecución de un script en PySpark
===========================

### Ejecución de un programa Spark

#### Shells Python y Scala

-   Scala: `$ bin/spark-shell`

-   Python: `$ bin/pyspark`

-   IPython:
    `$ export PYSPARK_DRIVER_PYTHON=ipython; bin/pyspark`

-   IPython Notebook:
    `$ export PYSPARK_DRIVER_PYTHON=ipython; export PYSPARK_DRIVER_PYTHON_OPTS="notebook"; bin/pyspark`

#### Ejecución con `spark-submit`

-   Permite lanzar programas Spark a un cluster

-   Ejemplo:

    `$ bin/spark-submit --master yarn --deploy-mode cluster \  
     --py-files otralib.zip,otrofich.py \  
     --num-executors 10 --executor-cores 2 \  
     mi-script.py opciones_del_script`

#### Opciones de `spark-submit`

-   `master`: cluster manager a usar (opciones: `yarn`,
    `mesos://host:port`, `spark://host:port`, `local`)

-   `deploy-mode`: dos modos de despliegue

    -   `client`: ejecuta el driver en el nodo local

    -   `cluster`: ejecuta el driver en un nodo del cluster

-   `class`: clase a ejecutar (Java o Scala)

-   `name`: nombre de la aplicación (se muestra en el Spark web)

-   `jars`: ficheros jar a añadir al classpath (Java o Scala)

-   `py-files`: archivos a añadir al PYTHONPATH (`.py`,`.zip`,`.egg`)

-   `files`: ficheros de datos para la aplicación

-   `executor-memory`: memoria total de cada ejecutor

-   `driver-memory`: memoria del proceso driver

Para más opciones: `spark-submit --help`

### Parámetros de configuración

Diversos parámetros ajustables en tiempo de ejecución

-   En un script

```python
from pyspark import SparkConf,SparkContext
conf = SparkConf()
conf.set("spark.app.name", "Mi apli")
conf.set("spark.master", "local[2]") # 2 cores
conf.set("spark.ui.port", "3600")    # Defecto: 4040
sc = SparkContext(conf=conf)
```

-   Mediante flags en el `spark-submit`

    `$ bin/spark-submit --master local[2] --name "Mi apli" \  
    --conf spark.ui.port=3600 mi-script.py`
    
    
-   Mediante un fichero de propiedades
    
    `$ cat config.conf 
    spark.master     local[2] 
    spark.app.name   "Mi apli" 
    spark.ui.port 3600`
    
    `$ bin/spark-submit --properties-file config.conf mi-script.py`

Más info:
<http://spark.apache.org/docs/latest/configuration.html#spark-properties>


### Trabajos, etapas y tareas
-   Un programa Spark define un DAG de RDDs

    -   Las transformaciones crean RDDs hijos a partir de RDDs padres

-   Las acciones traducen el DAG en un plan de ejecución

    -   El driver envía un *trabajo* (job) para computar todos los RDDs
        implicados en la acción

    -   El job se descompone en una o más *etapas* (stages)

    -   Cada stage corresponde a uno o más RDDs del DAG (*pipelining*)

    -   Las stages se procesan en orden, lanzándose *tareas* (tasks)
        individuales que computan segmentos de los RDDs

-   Pipelining: varios RDDs se computan en una misma stage

    -   Si los RDDs se pueden obtener de sus padres sin movimiento de
        datos (p.e. *map*)

    -   Si un RDD se ha *cacheado* en memoria o disco

    -   En el [interfaz web de Spark](http://localhost:4040 "PySpark en localhost") se muestran el número de stages por
        job (más info: método `toDebugString()` de los RDDs)

In [ ]:
from __future__ import print_function

rdd = sc.parallelize(xrange(1000)).cache()
rdd2 = (rdd.map(lambda x: (x, 2*x))
           .map(lambda (x,y): (x-100, y**2))
           .reduceByKey(lambda x,y: x+y)
           .values())
r = rdd2.reduce(lambda x,y:x+y)
print(rdd2.toDebugString())

## Práctica 5

Objetivo:

Desarrollar un script Pyspark, que haga dos operaciones, a partir de los ficheros cite75_99.txt y los ficheros secuence creados en la práctica anterior (a partir del fichero apat63_99.txt)

1. Realizad un Join similar al de Hadoop, para obtener, para cada patente, el país y el número de citas.
2. A partir de la información de las patentes (fichero sequence), obtener una salida de la forma:

    <tt>país     año -> no de patentes ese año; año -> no de patentes ese año; ....</tt>
    
    por ejemplo:
    
    <tt>ES      1963->26;1964->19;1965->49;...</tt>
    
    La salida debe estar ordenada por países, y para cada país por año.

Tened en cuenta lo siguiente:

- Se debe de crear un único script que haga los dos apartados en orden (usando persistencia de datos).
- En el Join realizad un join completo (full outer).
- Los datos de salida deben almacenarse  en ficheros de texto, en dos directorios separados, uno por apartado.

Plantilla a utilizar
```python
# -*- coding: utf-8 -*-

from __future__ import print_function
import sys
from pyspark import SparkContext,StorageLevel
#
# Fichero spark que implementa el codigo SimpleReduceSideJoin y sortsecundario2
# Entrada: fichero cite75_99.txt y el/los ficheros sequence creados en la práctica creasequencefile
#
# Ejecutar en local con:
# spark-submit --master local[*] sparkpractica.py dir-cite75_99.txt dir-apat63_99.seq outdir1 outdir2
#
# Y en el cluster
# spark-submit --master yarn sparkpractica.py dir-cite75_99.txt dir-apatseq outdir1 outdir2

def main():
    # Parámetros de entrada:
    # argv[1] = PATH al fichero cite75_99.txt
    # argv[2] = PATH a los ficheros sequence creados en la practica anterior
    # argv[3] = directorio de salida de Join
    # argv[4] = directorio de salida de Sort
    if len(sys.argv) != 5:
        print("Usage: sparkjoin.py file_citas.txt file(s)_apatseq outdir1 outdir2", file=sys.stderr)
        exit(-1)
    
    sc = SparkContext(appName="Spark practica")
    
    # Lee como un RDD el fichero cite75_99.txt
    citas = sc.textFile(sys.argv[1])

    # Elimina la cabecera y separa cada línea por la coma
    # TODO: Para cada patente citada genera un par clave/valor (patente,1) y acumula los 1s para cada clave
    num_citas = 
    
    # num_citas tiene el numero de veces que se ha citado una patente

    # Cargo los ficheros sequence (clave=país valor=n_patente,año ambos de tipo Text)
    info = 
    
    # TODO: Cachearlo porque lo voy a usar dos veces
    info.
    
    # TODO: Crea un RDD con clave el n_patente y valor el país
    pat_info = 
    
    # Hago un full outer join de num_citas y pat_info
    pat_country = 
    
    # Salvo en outdir1
    pat_country.saveAsTextFile(sys.argv[3])

    # La parte de sort (pais   año->n_patentes;año->n_patentes;...)
    # Primero coge el fichero info y hace el map: (pais patente,año) -> (pais,año 1)
    # luego reduce sumando los 1 y ordena por clave (para tener los años ordenados)
    # el siguiente map hace: (pais,año n) -> (pais año->n)
    # luego reduce por país para obtener (pais anho1->n1;anho2->n2;...)
    # y ordena otra vez por países
    country_year =     
    # ¿Hace esto lo que queremos?

    # salvo en outdir2
    country_year.saveAsTextFile(sys.argv[4])
    
if __name__ == "__main__":
    main()
```